In [3]:
from common import *
from gensim.models.wrappers import FastText

import io

In [45]:
# with io.open('../data/corpus_raw.txt', 'r', encoding='utf-8') as fr:
#     with io.open('../data/_corpus_raw.txt', 'w', encoding='utf-8') as fw:
#         for line in fr:
#             new_line = tokenize(line.lower())
#             if len(new_line):
#                 fw.write(' '.join(new_line) + u'\n')

In [1]:
# model = FastText.train('/root/fastText/fasttext', '../data/_corpus_raw.txt', 
#                        size=100, 
#                        model='skipgram',
#                        window=5, 
#                        min_count=5, 
#                        iter=5,
#                        threads=cpu_count)

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('../data/ft_output.vec')

2017-10-03 11:31:36,971 [MainThread  ] [INFO ]  loading projection weights from ../data/ft_output.vec
2017-10-03 11:42:32,561 [MainThread  ] [INFO ]  loaded (3648917, 100) matrix from ../data/ft_output.vec


In [6]:
most = model.most_similar(u'стол')
for w, score in most:
    print(w + ', %s' % score)

2017-10-03 11:42:32,580 [MainThread  ] [INFO ]  precomputing L2-norms of word weight vectors


стол,, 0.882844150066
столик, 0.872067809105
стол-рольганг, 0.834802746773
столом, 0.83262771368
стол-штатив, 0.828676521778
стола, 0.813986182213
стол-лоток, 0.813705742359
стол., 0.810664772987
стол-стеллаж, 0.808469176292
стол), 0.803629040718


In [7]:
with open(join(DATA_FOLDER, 'gold_mongo.json'), 'r') as f:
    gold = json.load(f)

In [10]:
def predict(keys, lim=200):
    names = pd.Series(range(len(fnames)), index = fnames)
    test_vecs = []
    for k in keys:
        ix = names[k]
        if isinstance(ix, pd.Series):
            ix = ix[0]
        tvec = doc_embeds_reshaped[ix]
        test_vecs.append(tvec)

    dists = euclidean_distances(test_vecs, doc_embeds_reshaped)
    sorted_ixs = np.argsort(dists, axis=1)
    preds = {}
    for k, _ixs in zip(keys, sorted_ixs):
        preds[k] = [n for n in fnames[_ixs[1:lim+1]]]
        
    return preds

In [11]:
preds = predict(gold.keys(), lim=200)

NameError: global name 'fnames' is not defined

In [ ]:
result = evaluate(preds, gold)

ax = result['acc10'].hist()
ax.set_xlabel("acc10")
plt.show()

ax = result['acc20'].hist()
ax.set_xlabel("acc20")
plt.show()

ax = result['acc200'].hist()
ax.set_xlabel("acc200")
plt.show()